    Roman Peerboom  10791523
    Floris de Vries 11710799

In [85]:
import numpy
import simpy
import random
import statistics 

In [236]:
lambda_1 = arrival_rate = 10              # Arrival rate, events in given time period
time_in_system = 20                       # Mean time in system 
mu = 1/time_in_system                     # How much people one server can help for a given time period 


In [392]:
def source(env, number, lambda_1, counter, mu):
    """Source generates customers randomly"""
    for i in range(number):
        c = object(env, 'Customer%02d' % i, counter, mu)
        env.process(c)
        t = 1-random.expovariate(-lambda_1)
        yield env.timeout(t)
        
        
def object(env, name, counter, mu):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))
    with counter.request() as req:
        results = yield req
        wait = env.now - arrive
        # We got to the counter
        print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        tib = 1- random.expovariate(-mu)
        yield env.timeout(tib)
        print('%7.4f %s: Finished' % (env.now, name))


lambda_1 = arrival_rate = 2                # Arrival rate, events in given time period
time_in_system = 0.5                       # Mean time in system 
mu = 1/time_in_system                      # How much people one server can help for a given time period 
number_of_arrivals = 10000000000
print("\N{greek small letter rho} is: " + str(lambda_1/mu))

env = simpy.Environment()
server = simpy.Resource(env, capacity=1) 
env.process(source(env,number_of_arrivals,lambda_1, server, mu))
env.run(until=1000)


ρ is: 1.0
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 1.0847 Customer01: Here I am
 1.8351 Customer00: Finished
 1.8351 Customer01: Waited  0.750
 2.4769 Customer02: Here I am
 2.9865 Customer01: Finished
 2.9865 Customer02: Waited  0.510
 3.4993 Customer03: Here I am
 4.0777 Customer02: Finished
 4.0777 Customer03: Waited  0.578
 5.2501 Customer04: Here I am
 5.6052 Customer03: Finished
 5.6052 Customer04: Waited  0.355
 6.6270 Customer04: Finished
 6.7822 Customer05: Here I am
 6.7822 Customer05: Waited  0.000
 7.7885 Customer05: Finished
 8.0340 Customer06: Here I am
 8.0340 Customer06: Waited  0.000
 9.0932 Customer07: Here I am
 9.1357 Customer06: Finished
 9.1357 Customer07: Waited  0.042
10.5045 Customer07: Finished
12.7418 Customer08: Here I am
12.7418 Customer08: Waited  0.000
13.9198 Customer09: Here I am
14.6041 Customer08: Finished
14.6041 Customer09: Waited  0.684
15.9167 Customer09: Finished
17.8648 Customer10: Here I am
17.8648 Customer10: Waited  0.

In [391]:
t_list = []
t_leave_list = []

for i in range(10000):
    t = 1-random.expovariate(-lambda_1)
    l = 1-random.expovariate(-mu)
    t_list.append(t)
    t_leave_list.append(l)

print(lambda_1)
print(mu)
print(statistics.mean(t_list))
print(statistics.mean(t_leave_list))

2
2.0
1.512638699056214
1.4974083881405271


In [ ]:
""" bank08: A counter with a random service time """
from SimPy.Simulation import *
from random import expovariate, seed

## Model components ------------------------           

class Source(Process):
    """ Source generates customers randomly """

    def generate(self,number,meanTBA,resource):         
        for i in range(number):
            c = Customer(name = "Customer%02d"%(i,))
            activate(c,c.visit(b=resource))              
            t = expovariate(1.0/meanTBA)               
            yield hold,self,t

class Customer(Process):
    """ Customer arrives, is served and leaves """
        
    def visit(self,b):                                
        arrive = now()
        print "%8.4f %s: Here I am     "%(now(),self.name)
        yield request,self,b                          
        wait = now()-arrive
        print "%8.4f %s: Waited %6.3f"%(now(),self.name,wait)
        tib = expovariate(1.0/timeInBank)            
        yield hold,self,tib                          
        yield release,self,b                         
        print "%8.4f %s: Finished      "%(now(),self.name)

## Experiment data -------------------------         

maxNumber = 5
maxTime = 400.0 # minutes                                     
timeInBank=12.0 # mean, minutes                      
ARRint = 10.0   # mean, minutes                      
theseed= 12345                                       

## Model/Experiment ------------------------------

seed(theseed)                                        
k = Resource(name="Counter",unitName="Clerk")       

initialize()
s = Source('Source')
activate(s,s.generate(number=maxNumber,meanTBA=ARRint, 
                      resource=k),at=0.0)           
simulate(until=maxTime)

In [ ]:
""" bank09: Several Counters but a Single Queue """
from SimPy.Simulation import *
from random import expovariate, seed

## Model components ------------------------           

class Source(Process):
    """ Source generates customers randomly """

    def generate(self,number,meanTBA,resource):      
        for i in range(number):
            c = Customer(name = "Customer%02d"%(i,))
            activate(c,c.visit(b=resource))            
            t = expovariate(1.0/meanTBA)              
            yield hold,self,t


class Customer(Process):
    """ Customer arrives, is served and leaves """
        
    def visit(self,b):                                
        arrive = now()
        print "%8.4f %s: Here I am     "%(now(),self.name)
        yield request,self,b                          
        wait = now()-arrive
        print "%8.4f %s: Waited %6.3f"%(now(),self.name,wait)
        tib = expovariate(1.0/timeInBank)            
        yield hold,self,tib                          
        yield release,self,b                         
        print "%8.4f %s: Finished      "%(now(),self.name)
        
class Customer(Process):
    """ Customer arrives, is served and leaves """
        
    def visit(self,b):                                
        arrive = now()
        print "%8.4f %s: Here I am     "%(now(),self.name)
        yield request,self,b                          
        wait = now()-arrive
        print "%8.4f %s: Waited %6.3f"%(now(),self.name,wait)
        tib = expovariate(1.0/timeInBank)            
        yield hold,self,tib                          
        yield release,self,b                         
        print "%8.4f %s: Finished      "%(now(),self.name)


## Experiment data -------------------------         

maxNumber = 5
maxTime = 400.0 # minutes                                      
timeInBank = 12.0 # mean, minutes                      
ARRint = 10.0   # mean, minutes                      
theseed = 12345                                        

## Model/Experiment ------------------------------

seed(theseed)                                        
k = Resource(capacity=2,name="Counter",unitName="Clerk")  

initialize()
s = Source('Source')
activate(s, s.generate(number=maxNumber,meanTBA=ARRint, 
                         resource=k),at=0.0)           
simulate(until=maxTime)